# Covid19 Food Insecurity Estimates

SJ Terp
2020-03-27
Version 4.0

We were asked to look for which countries might need a "food airlift" during the Covid19 pandemic.  There are many reasons this might happen, for example sick farmers and fishermen unable to tend crops and livestock, or fish from boats; agricultural distribution issues in and between countries when supply-chains are focussed on medical and essentials etc.  The issue that this notebook takes a (very rough) look at is the effect on food supply chains and food security of countries if country borders are closed, e.g. if imports and exports of food stop or dwindle to a low level. 

There are longer notes about this.  The TL;DR is that one place to start with these calculations is with existing calculations of food security, and remove the import and export components of them.  A good first source is the FAO, which maintains "Food Balance Sheets" for 209 countries and regions of the world at http://www.fao.org/faostat/en/#data/FBS. Reading the web notes ('definitions and standards') for this dataset, we see that if we download the dataset, the number we want for each country is (Item Code = 2901, Element Code = 664), aka the Grand Total for each country of its Food supply in kilocalories per capita per day (e.g. the Grand Total for Canada is around 3000 kcal available per person per day).  

The country Grand Total kcal/capita/day is the sum of kcal/capita/day across a set of items (e.g. Meat), each of which is the sum of a lower level of items (e.g. Pigmeat, Poultry etc).  The web notes for the dataset also contains an 'Item Group' list describing this structure, which we'll use in our calculations.

Timewise, the FAO Food Balance Sheets go up to 2017. That's recent enough for a rough calculation. The FAO has also left enough raw data in the balance sheets for us to do the rough recalculations that we need. 

Assumptions and caveats
* This dataset only covers 207 countries and areas of the world. Action: Check to see if other countries are left out, and if they are, how we could proxy their figures
* These approximations are based on 2017 numbers. They're not that old, but climate effects on crops have moved quickly in the past couple of years. 
* We're only calculating using the top level of the Item Group hierachy. We're doing rough calculations, so doing them over all possible items seemed a little over the top. 
* ACTION: Usually food security calculations average over 3 years to get a smoothed annual score, so should do that here too. 
* Some of the original numbers look... a little healthier than expected? Might need to be cautious with any self-reported data. 

Fixits:

## Get dataset

The dataset we're working with is the FAO's New Food Balances, available at http://www.fao.org/faostat/en/#data/FBS

* Get the main dataset by clicking "all data" under "bulk downloads" on http://www.fao.org/faostat/en/#data/FBS

We're going to grab everything immediately below the 'Grand Total' code (2901) in the structure, and throw it into a dataframe called dfc, and an array called cats. 

* Get the item group codes dataset by clicking "definitions and standards" then "item groups" then the download symbol on http://www.fao.org/faostat/en/#data/FBS
* create a list of top-level categories by filtering for items with parent category 2901 ("Item group" = "grand total")

In [1]:
import pandas as pd

# Import the list of top-level food categories (these are the ones we'll adapt and sum over)
dfcats = pd.read_csv('../DATA/FAOSTAT_data_3-24-2020_itemtotals.csv')
dfc = dfcats[dfcats['Item Group Code']==2901][['Item Code', 'Item', 'Factor']]
cats = dfc['Item Code'].to_list()

# Import the food balance sheets for 2014-2018
df = pd.read_csv('../DATA/FoodBalanceSheets_E_All_Data/FoodBalanceSheets_E_All_Data.csv',
                 encoding='iso-8859-1')
df.head(3)

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1000 persons,33370.79,S,34413.6,S,35383.03,S,36296.11,S
1,2,Afghanistan,2501,Population,5301,Domestic supply quantity,1000 tonnes,0.00,S,0.0,S,0.00,S,0.00,S
2,2,Afghanistan,2901,Grand Total,664,Food supply (kcal/capita/day),kcal/capita/day,2019.00,Fc,1978.0,Fc,2001.00,Fc,2000.00,Fc


## Reformat dataset for analysis work

The dataset as-is is going to be difficult to apply column-by-column manipulations to.  This section grabs only the top-level entries from the original datatable, and reformats it so it's easier to work with.   Yes, this could be done a lot more cleanly with a couple of pandas commands, but frankly we didn't have time.  In here: 

* dfi - reformatted dataset
* yrs = years of interest
* inds = columns we're keeping from the old table (that aren't annual numbers)
* els = table element_codes we want to keep in the table

In [2]:
#yrs = ['Y2014', 'Y2015', 'Y2016', 'Y2017']
yrs = ['Y2017']
inds = ['Area', 'Area Code', 'Item Code', 'Item']
els = [664, 5511, 5611, 5911, 5072, 5301, 5123, 645, 5521, 5527, 5131, 5154, 5171, 5170, 5142]

def mkdfel(df, el): 
    dfel = (df[(df['Element Code']==el)&(df['Item Code'].isin(cats))][inds+yrs]).copy()
    newcols = {col:'{}_{}'.format(col,el) for col in set(dfel.columns)-set(inds)}
    dfel.rename(columns=newcols, inplace=True)
    return dfel

elements = {}
for el in els: 
    thisel = mkdfel(df, el)
    if el == els[0]:
        dfi = thisel
    else:
        dfi = pd.merge(dfi, thisel, on=inds, how='outer')

# Small hack to add in columns of population numbers
dfel = df[df['Element Code'] == 511][['Area Code'] + yrs]
newcols = {col:'{}_{}'.format(col, 511) for col in yrs}
dfel.rename(columns=newcols, inplace=True)
dfi = pd.merge(dfi, dfel, on=['Area Code'], how='outer')
        
dfi.fillna(0, inplace=True)
dfi.head(10)

,Area,Area Code,Item Code,Item,Y2017_664,Y2017_5511,Y2017_5611,Y2017_5911,Y2017_5072,Y2017_5301,Y2017_5123,Y2017_645,Y2017_5521,Y2017_5527,Y2017_5131,Y2017_5154,Y2017_5171,Y2017_5170,Y2017_5142,Y2017_511
0,Afghanistan,2,2511,Wheat and products,1331.0,4281.0,2302.0,0.0,-119.0,6701.0,642.0,155.39,76.0,344.0,0.0,0.0,0.0,0.0,5640.0,36296.11
1,Afghanistan,2,2805,Rice (Milled Equivalent),135.0,338.0,415.0,0.0,0.0,753.0,24.0,19.77,0.0,11.0,0.0,0.0,0.0,0.0,717.0,36296.11
2,Afghanistan,2,2513,Barley and products,5.0,95.0,15.0,0.0,0.0,110.0,14.0,0.51,67.0,11.0,0.0,0.0,0.0,0.0,18.0,36296.11
3,Afghanistan,2,2514,Maize and products,11.0,174.0,13.0,0.0,-21.0,208.0,26.0,1.38,111.0,21.0,0.0,0.0,0.0,0.0,50.0,36296.11
4,Afghanistan,2,2517,Millet and products,2.0,9.0,0.0,0.0,-1.0,10.0,0.0,0.26,0.0,0.0,0.0,0.0,0.0,0.0,9.0,36296.11
5,Afghanistan,2,2520,"Cereals, Other",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36296.11
6,Afghanistan,2,2531,Potatoes and products,13.0,513.0,230.0,5.0,414.0,324.0,22.0,6.29,0.0,73.0,0.0,0.0,0.0,0.0,228.0,36296.11
7,Afghanistan,2,2542,Sugar (Raw Equivalent),88.0,2.0,483.0,0.0,155.0,331.0,0.0,9.12,0.0,0.0,0.0,0.0,0.0,0.0,331.0,36296.11
8,Afghanistan,2,2543,"Sweeteners, Other",1.0,0.0,17.0,0.0,0.0,17.0,0.0,0.48,0.0,0.0,0.0,0.0,0.0,0.0,17.0,36296.11
9,Afghanistan,2,2745,Honey,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,1.0,36296.11


# Approximate each Food Supply with imports and exports removed

Last time, we used the ratio between original DSQ, and DSQ without Import/Export. This time, for each country and food group, we're approximating the calculation between FSQ and food supply.  The exact calculations used are:

* Lossfrac = L / (P + I + SV)   <- using the original numbers
* fsqtofoodsupply = (FoodSupply / FSQ) <- using the original numbers
* newDSQ = P + SV <- using the original numbers
* newlosses = newDSQ * lossfrac  
* newfood = newDSQ - (Feed + Seed + newlosses + Processing + Other_uses + Tourist + Residuals)
* newFSQ = newfood * 1000 / Population
* newFoodSupply = newFSQ * fsqtofoodsupply

Variables used from the original table are:
* L is Losses (element code 5123)
* P is Production (element code 5511)
* I is imports (element code 5611)
* SV is stock variation (element code 5072)
* Foodsupply (element code 664)
* FSQ is food supply quantity (element code 645)
* Feed (element code 5521)
* Seed (element code 5527)
* Processing (element code 5131)
* Other_users (element code 5154)
* Tourist (element code 5171)
* Residuals (element code 5170)
* Population (element code 511)

Once that's done: 
* Summing the FSQ for each country
* Look at the differences. Plot them? Think about which foods are continuously-gathered, seasonal etc. 

Approximations made here: 
* We're approximating FSQ to Food Supply, using the existing nubmers to get a ratio for each food and country.  The real tables for this are... painful. 
* We've left the tourists in the tables for now.  We can make assumptions about them (e.g. there won't be tourists, or there will be extra population because people fled to their island holiday homes etc) later
* We've left the residuals as-is, even though we know they're calculated in some dystopian way
* This isn't taking any account of negative food.  If there isn't food available to feed tourists, we ignore this: we allow foods to go negative. 
* About that negative food.  We don't, however, allow the food available (5142) to go below zero, because if it does it's one hell of a mess.  What this practically means is that we're not allowing for cases like that people might not be able to feed animals (5521) or keep back seed for next year (5527). 

In [3]:
import numpy as np
dfi['Y2017_lossfrac'] = dfi['Y2017_5123'] / (dfi['Y2017_5511'] + dfi['Y2017_5611'] + dfi['Y2017_5072'])
dfi['Y2017_fsq2fs'] = dfi['Y2017_664'] / dfi['Y2017_645'] 
dfi.loc[dfi['Y2017_fsq2fs'] == np.inf, 'Y2017_fsq2fs'] = 0 #handle divide by zero from rows with 0 FSQ
dfi['Y2017_newDSQ'] = dfi['Y2017_5511'] + dfi['Y2017_5072']
dfi['Y2017_newlosses'] = dfi['Y2017_newDSQ'] * dfi['Y2017_lossfrac']
dfi['Y2017_newfood'] = dfi['Y2017_newDSQ'] - (dfi['Y2017_5521'] + dfi['Y2017_5527'] + dfi['Y2017_newlosses'] + 
                                              dfi['Y2017_5131'] + dfi['Y2017_5154'] + dfi['Y2017_5171'] + 
                                              dfi['Y2017_5170'])
dfi.loc[dfi['Y2017_newfood'] < 0, 'Y2017_newfood'] = 0 #don't have negative food
dfi['Y2017_newFSQ'] = dfi['Y2017_newfood'] * 1000.0 / dfi['Y2017_511']
dfi['Y2017_newFoodSupply'] = dfi['Y2017_newFSQ'] * dfi['Y2017_fsq2fs']
dfi.fillna(0, inplace=True)

dfi[inds +['Y2017_newFoodSupply', 'Y2017_664', 'Y2017_newDSQ', 'Y2017_5301', 
           'Y2017_newlosses', 'Y2017_5123', 'Y2017_lossfrac', 'Y2017_newfood', 'Y2017_5142', 
           'Y2017_newFSQ', 'Y2017_645', 'Y2017_fsq2fs']]#.sort_values('Y2017_newFoodSupply', ascending=False)

,Area,Area Code,Item Code,Item,Y2017_newFoodSupply,Y2017_664,Y2017_newDSQ,Y2017_5301,Y2017_newlosses,Y2017_5123,Y2017_lossfrac,Y2017_newfood,Y2017_5142,Y2017_newFSQ,Y2017_645,Y2017_fsq2fs
0,Afghanistan,2,2511,Wheat and products,785.526475,1331.0,4162.0,6701.0,413.366955,642.0,0.099319,3328.633045,5640.0,91.707708,155.39,8.565545
1,Afghanistan,2,2805,Rice (Milled Equivalent),59.493030,135.0,338.0,753.0,10.772908,24.0,0.031873,316.227092,717.0,8.712424,19.77,6.828528
2,Afghanistan,2,2513,Barley and products,1.325992,5.0,95.0,110.0,12.090909,14.0,0.127273,4.909091,18.0,0.135251,0.51,9.803922
3,Afghanistan,2,2514,Maize and products,0.000000,11.0,153.0,208.0,23.963855,26.0,0.156627,0.000000,50.0,0.000000,1.38,7.971014
4,Afghanistan,2,2517,Millet and products,1.695456,2.0,8.0,10.0,0.000000,0.0,0.000000,8.000000,9.0,0.220409,0.26,7.692308
5,Afghanistan,2,2520,"Cereals, Other",0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000
6,Afghanistan,2,2531,Potatoes and products,47.624763,13.0,927.0,324.0,17.626621,22.0,0.019015,836.373379,228.0,23.043058,6.29,2.066773
7,Afghanistan,2,2542,Sugar (Raw Equivalent),41.737593,88.0,157.0,331.0,0.000000,0.0,0.000000,157.000000,331.0,4.325532,9.12,9.649123
8,Afghanistan,2,2543,"Sweeteners, Other",0.000000,1.0,0.0,17.0,0.000000,0.0,0.000000,0.000000,17.0,0.000000,0.48,2.083333
9,Afghanistan,2,2745,Honey,0.000000,0.0,1.0,2.0,0.000000,0.0,0.000000,1.000000,1.0,0.027551,0.04,0.000000


## Double-check interim results

Just a geek sanity-check: putting the old and new column values next to each other for a visual check. 

In [4]:
testcountry = 'Peru'
pd.set_option('display.max_columns', 1000)
dfi[dfi['Area']==testcountry].sort_values('Y2017_newFoodSupply', ascending=False)

,Area,Area Code,Item Code,Item,Y2017_664,Y2017_5511,Y2017_5611,Y2017_5911,Y2017_5072,Y2017_5301,Y2017_5123,Y2017_645,Y2017_5521,Y2017_5527,Y2017_5131,Y2017_5154,Y2017_5171,Y2017_5170,Y2017_5142,Y2017_511,Y2017_lossfrac,Y2017_fsq2fs,Y2017_newDSQ,Y2017_newlosses,Y2017_newfood,Y2017_newFSQ,Y2017_newFoodSupply
11592,Peru,170,2805,Rice (Milled Equivalent),544.0,3039.0,566.0,0.0,16.0,3589.0,62.0,76.44,18.0,23.0,115.0,968.0,0.0,0.0,2403.0,31444.3,0.017122,7.116693,3055.0,52.308754,1878.691246,59.746639,425.198479
11671,Peru,170,2763,Pelagic Fish,33.0,3615.0,155.0,4444.0,1122.0,448.0,0.0,13.98,8.0,0.0,0.0,0.0,0.0,0.0,439.0,31444.3,0.000000,2.360515,4737.0,0.000000,4729.000000,150.392917,355.004740
11603,Peru,170,2542,Sugar (Raw Equivalent),215.0,1081.0,583.0,133.0,269.0,1262.0,28.0,22.09,0.0,0.0,250.0,289.0,0.0,0.0,695.0,31444.3,0.014485,9.732911,1350.0,19.555096,791.444904,25.169742,244.974850
11599,Peru,170,2531,Potatoes and products,222.0,4776.0,49.0,0.0,0.0,4825.0,1109.0,82.58,475.0,388.0,0.0,257.0,0.0,0.0,2597.0,31444.3,0.229845,2.688302,4776.0,1097.737617,2558.262383,81.358541,218.716350
11598,Peru,170,2532,Cassava and products,127.0,1196.0,6.0,16.0,0.0,1186.0,172.0,28.60,106.0,0.0,6.0,4.0,0.0,0.0,899.0,31444.3,0.143095,4.440559,1196.0,171.141431,908.858569,28.903762,128.348874
11668,Peru,170,2848,Milk - Excluding Butter,103.0,2035.0,59.0,190.0,0.0,1904.0,41.0,56.68,80.0,0.0,0.0,8.0,0.0,-8.0,1782.0,31444.3,0.019580,1.817219,2035.0,39.844795,1915.155205,60.906276,110.680071
11639,Peru,170,2616,Plantains,102.0,2026.0,0.0,0.0,0.0,2026.0,612.0,44.95,0.0,0.0,0.0,0.0,0.0,0.0,1413.0,31444.3,0.302073,2.269188,2026.0,612.000000,1414.000000,44.968404,102.041763
11624,Peru,170,2577,Palm Oil,64.0,160.0,5.0,42.0,0.0,123.0,0.0,2.64,0.0,0.0,0.0,40.0,0.0,0.0,83.0,31444.3,0.000000,24.242424,160.0,0.000000,120.000000,3.816272,92.515684
11644,Peru,170,2625,"Fruits, Other",37.0,1464.0,49.0,529.0,0.0,983.0,213.0,25.92,0.0,0.0,7.0,0.0,0.0,-51.0,815.0,31444.3,0.140780,1.427469,1464.0,206.101785,1301.898215,41.403314,59.101952
11659,Peru,170,2734,Poultry Meat,62.0,1465.0,60.0,1.0,0.0,1523.0,316.0,13.47,0.0,0.0,0.0,783.0,0.0,0.0,424.0,31444.3,0.207213,4.602821,1465.0,303.567213,378.432787,12.035020,55.395045


In [5]:
dfi[dfi['Area']==testcountry][inds +['Y2017_newFoodSupply', 'Y2017_664', 'Y2017_newDSQ', 'Y2017_5301', 
           'Y2017_newlosses', 'Y2017_5123', 'Y2017_lossfrac', 'Y2017_newfood', 'Y2017_5142', 
           'Y2017_newFSQ', 'Y2017_645', 'Y2017_fsq2fs']].sort_values('Y2017_newFoodSupply')

,Area,Area Code,Item Code,Item,Y2017_newFoodSupply,Y2017_664,Y2017_newDSQ,Y2017_5301,Y2017_newlosses,Y2017_5123,Y2017_lossfrac,Y2017_newfood,Y2017_5142,Y2017_newFSQ,Y2017_645,Y2017_fsq2fs
11688,Peru,170,2659,"Alcohol, Non-Food",0.000000,0.0,29.0,93.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000
11621,Peru,170,2574,Rape and Mustard Oil,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000
11682,Peru,170,2518,Sorghum and products,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000
11625,Peru,170,2578,Coconut Oil,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000
11626,Peru,170,2579,Sesameseed Oil,0.000000,1.0,0.0,1.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.03,33.333333
11628,Peru,170,2581,Ricebran Oil,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000
11629,Peru,170,2582,Maize Germ Oil,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000
11676,Peru,170,2768,"Meat, Aquatic Mammals",0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000
11636,Peru,170,2613,Grapefruit and products,0.000000,0.0,7.0,6.0,1.000000,1.0,0.142857,6.000000,5.0,0.190814,0.15,0.000000
11687,Peru,170,2562,Palm kernels,0.000000,0.0,852.0,852.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000


In [6]:
df[(df['Area']==testcountry) & (df['Item Code']==2578)][['Element Code', 'Element', 'Unit', 'Y2016', 'Y2017']]

,Element Code,Element,Unit,Y2016,Y2017
182469,5511,Production,1000 tonnes,0.00,0.00
182470,5611,Import Quantity,1000 tonnes,0.00,0.00
182471,5072,Stock Variation,1000 tonnes,0.00,NaN
182472,5911,Export Quantity,1000 tonnes,0.00,0.00
182473,5301,Domestic supply quantity,1000 tonnes,0.00,0.00
182474,5521,Feed,1000 tonnes,0.00,NaN
182475,5527,Seed,1000 tonnes,0.00,NaN
182476,5123,Losses,1000 tonnes,0.00,NaN
182477,5131,Processing,1000 tonnes,0.00,NaN
182478,5154,Other uses (non-food),1000 tonnes,0.00,NaN


In [7]:
dfi[dfi['Area']==testcountry].describe()

,Area Code,Item Code,Y2017_664,Y2017_5511,Y2017_5611,Y2017_5911,Y2017_5072,Y2017_5301,Y2017_5123,Y2017_645,Y2017_5521,Y2017_5527,Y2017_5131,Y2017_5154,Y2017_5171,Y2017_5170,Y2017_5142,Y2017_511,Y2017_lossfrac,Y2017_fsq2fs,Y2017_newDSQ,Y2017_newlosses,Y2017_newfood,Y2017_newFSQ,Y2017_newFoodSupply
count,98.0,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.0,98.000000,98.000000,9.800000e+01,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,170.0,2629.489796,28.234694,450.877551,89.377551,82.336735,3.020408,478.989796,45.306122,7.100000,52.020408,5.755102,119.877551,33.540816,0.0,-0.816327,223.234694,3.144430e+04,0.049566,5.087601,453.897959,42.175118,262.820756,8.358296,24.773194
std,0.0,92.194577,76.488231,1208.112076,412.057836,453.589417,179.710820,1361.504175,147.471640,16.033659,400.673093,39.630786,1022.737909,134.640581,0.0,5.498541,504.126616,4.022352e-11,0.076003,7.616553,1149.790886,145.678241,659.650927,20.978394,66.239598
min,170.0,2511.000000,0.000000,0.000000,0.000000,0.000000,-1333.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-51.000000,0.000000,3.144430e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,170.0,2557.250000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.144430e+04,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,170.0,2611.500000,3.000000,42.000000,1.500000,0.000000,0.000000,40.500000,0.000000,0.725000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,22.500000,3.144430e+04,0.000000,1.422954,45.500000,0.000000,27.000000,0.858661,2.692980
75%,170.0,2731.750000,16.750000,216.500000,16.000000,28.750000,0.000000,240.500000,15.750000,4.262500,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,134.000000,3.144430e+04,0.076571,8.569932,239.000000,14.500000,156.783149,4.986059,15.755541
max,170.0,2899.000000,544.000000,9400.000000,3397.000000,4444.000000,1122.000000,10733.000000,1109.000000,82.580000,3945.000000,388.000000,10114.000000,968.000000,0.0,0.000000,2597.000000,3.144430e+04,0.333333,33.333333,8067.000000,1097.737617,4729.000000,150.392917,425.198479


## Produce food supply estimate for each country

Create a rough estimate of dietary energy supply (kcal/cap/day) for each country by summing the new product calories for that country. Compare against the old calorie estimates. 

In [35]:
pd.set_option('display.max_rows', 300)
des_all = dfi[['Area', 'Y2017_664', 'Y2017_newFoodSupply']].groupby('Area').sum().sort_values('Y2017_newFoodSupply')
des_all['percent'] = 100.0 * des_all['Y2017_newFoodSupply'] / 1200.0
des_all.rename(columns = {'Y2017_664': 'FAO 2017 estimate', 
                      'Y2017_newFoodSupply': 'FAO 2017 without imports, exports', 
                      'percent': 'percent of original'},
                    inplace=True)

# ACTION: Edit country names so they get through Tableau's filters
# Current issue: Tableau doesn't recognise Taiwan

des_all.to_csv('caloriescapitaday.csv')
des_all.sort_values('FAO 2017 without imports, exports', ascending=True)

,FAO 2017 estimate,"FAO 2017 without imports, exports",percent of original
Area,,,
Bermuda,0.0,0.000000,0.000000
Brunei Darussalam,0.0,0.000000,0.000000
"China, Hong Kong SAR",3353.0,327.178642,27.264887
Antigua and Barbuda,2429.0,330.530207,27.544184
New Caledonia,2750.0,395.399293,32.949941
Yemen,2062.0,522.354795,43.529566
Jordan,2716.0,534.987477,44.582290
Cabo Verde,2512.0,558.519855,46.543321
"China, Macao SAR",2999.0,593.961212,49.496768


In [50]:
# Create a clean version for Tableau use
import numpy as np
dfcountries = df[['Area Code', 'Area']].drop_duplicates()
dfcountries = dfcountries[dfcountries['Area Code'] < 5000]
des_countries = pd.merge(des_all, dfcountries, left_index=True, right_on='Area')
des_countries.rename(columns = {'percent': 'percent of original', 
                      'Area Code':'FAO Country Code', 
                      'Area': 'Country name'},
                    inplace=True)
des_countries['FAO 2017 without imports, exports'] = des_countries['FAO 2017 without imports, exports'].round()
des_countries['FAO 2017 without imports, exports, rounded'] = 100*(des_countries['FAO 2017 without imports, exports']/100.0).apply(np.floor)
des_countries['percent of original'] = des_countries['percent of original'].round()
des_countries['oldstatus'] = pd.cut(des_countries['FAO 2017 estimate'], 
                                    [0, 1200, 1800, 1000000], right=False,
                                    labels=["below 1200", "below 1800", "above 1800"])
des_countries['newstatus'] = pd.cut(des_countries['FAO 2017 without imports, exports'], 
                                    [0, 1200, 1800, 1000000], right=False,
                                    labels=["below 1200", "below 1800", "above 1800"])
des_countries.to_csv('caloriescapitaday_for_tableau.csv', index=False)
des_countries

,FAO 2017 estimate,"FAO 2017 without imports, exports",percent of original,FAO Country Code,Country name,"FAO 2017 without imports, exports, rounded",oldstatus,newstatus
24438,0.0,0.0,0.0,17,Bermuda,0.0,below 1200,below 1200
30517,0.0,0.0,0.0,26,Brunei Darussalam,0.0,below 1200,below 1200
47307,3353.0,327.0,27.0,96,"China, Hong Kong SAR",300.0,above 1800,below 1200
6237,2429.0,331.0,28.0,8,Antigua and Barbuda,300.0,above 1800,below 1200
165382,2750.0,395.0,33.0,153,New Caledonia,300.0,above 1800,below 1200
249586,2062.0,522.0,44.0,249,Yemen,500.0,above 1800,below 1200
123390,2716.0,535.0,45.0,112,Jordan,500.0,above 1800,below 1200
33869,2512.0,559.0,47.0,35,Cabo Verde,500.0,above 1800,below 1200
48785,2999.0,594.0,49.0,128,"China, Macao SAR",500.0,above 1800,below 1200
114606,2505.0,652.0,54.0,103,Iraq,600.0,above 1800,below 1200


In [46]:
print('Num countries: {}'.format(len(des_countries)))
print('Under 1000 kcal/capita/day: {}'.format(len(des_countries[des_countries['FAO 2017 without imports, exports']<=1000])))
print('Under 1200 kcal/capita/day: {}'.format(len(des_countries[des_countries['FAO 2017 without imports, exports']<=1200])))
print('Under 1800 kcal/capita/day: {}'.format(len(des_countries[des_countries['FAO 2017 without imports, exports']<=1800])))

Num countries: 175
Under 1000 kcal/capita/day: 28
Under 1200 kcal/capita/day: 37
Under 1800 kcal/capita/day: 60


In [51]:
des_countries.describe()

,FAO 2017 estimate,"FAO 2017 without imports, exports",percent of original,FAO Country Code,"FAO 2017 without imports, exports, rounded"
count,175.000000,175.000000,175.000000,175.000000,175.000000
mean,2827.628571,3357.845714,279.880000,126.720000,3309.714286
std,544.651206,3253.073690,271.078926,75.168519,3253.127130
min,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2508.500000,1348.000000,112.500000,64.500000,1300.000000
50%,2871.000000,2428.000000,202.000000,121.000000,2400.000000
75%,3245.500000,3748.000000,312.000000,188.500000,3700.000000
max,3770.000000,18530.000000,1544.000000,351.000000,18500.000000
